In [ ]:
%load_ext autoreload
%autoreload 2

# Working with IMAS files

This example shows how *duqtools* can be used to manipulate IMAS data using `ImasHandle` and `IDSMapping`.

## Copying data

This example shows how *duqtools* can be used to copy data.

In [ ]:
from duqtools.api import ImasHandle

First, we construct and Imas handle, consisting of the user name, database (sometimes referred to as machine name), shot and run number. Together these form an Imas path that point to some data on the system.

We can print the path, and check that the data exists.

Note that the user is optional. Duqtools will fill in the current user by default.

In [ ]:
handle = ImasHandle(user='g2ssmee', db='jet', shot=94875, run=8000)
print(handle.path())
handle.exists()

Let's say we want to make a copy of this data set to modify. We should construct a new Imas handle to specify the target location. In the next cell, we create the Imas handle from a string instead. Again, the user is optional, so the string `'jet/94875/8888'` would give the same result.

In [ ]:
target = ImasHandle.from_string('g2ssmee/jet/94785/8888')
print(target.path())
target.exists()

The template data can be copied to the new location.

In [ ]:
handle.copy_data_to(target)
target.exists()

If you are unhappy with the outcome, wecan delete the data using the `ImasHandle.delete()` method. Let's not do that for now.

In [ ]:
# target.delete()
target.exists()

## Exploring, modifying, and saving data

We developed *duqtools* to make it as straightforward as possible to manipulate IDSs. In this section, we show an example of how one may go about doing so.

Let's say we want to look at 'core_profiles' > 'profiles_1d' > 't_i_average'.

First, we retrieve the 'core_profiles' IDS.

In [ ]:
cp = target.get('core_profiles')
cp

*Duqtools* wraps the IMAS database in a dict-like object, so that the different datasets can be accessed like a dict.

In [ ]:
cp['profiles_1d/0/t_i_average']

In [ ]:
cp['profiles_1d/0/t_i_average'] *= 1.1

Once we changed the data, we can sync the changes back to the data file.

In [ ]:
cp.sync(target)

## Plotting data

The example below shows how can plot some data using [matplotlib](https://matplotlib.org).

In [ ]:
x = cp['profiles_1d/0/grid/rho_tor_norm']
y = cp['profiles_1d/0/t_i_average']

import matplotlib.pyplot as plt

fig, ax = plt.subplots()

ax.plot(x, y)
ax.set_xlabel('rho_tor_norm')
ax.set_ylabel('t_i_average')